In [5]:
from elasticsearch7 import Elasticsearch
import os
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re

es = Elasticsearch("http://localhost:9200/")
directory_path = '../ap89_collection/'
query_path = '../AP_DATA/query_desc.51-100.short.txt'
index_name = 'ap89_collection'
stop_words_path = '../AP_DATA/stoplist.txt'
es.ping()

True

In [6]:
import os
import pandas as pd
import re


# Create a list to store the data
data = []

# Loop through all files in the directory
for filename in os.listdir(directory_path):
    # Construct the full path to the file
    file_path = os.path.join(directory_path, filename)

    with open(file_path, 'r', encoding='ISO-8859-1', errors='ignore') as file:
        content = file.read()

    # Use regular expression to extract each document
    docs = re.findall(r'<DOC>.*?</DOC>', content, flags=re.DOTALL)

    # Extract metadata and text for each document
    for doc in docs:
        docno_match = re.search(r'<DOCNO>(.*?)</DOCNO>', doc)
        fileid_match = re.search(r'<FILEID>(.*?)</FILEID>', doc)
        first_match = re.search(r'<FIRST>(.*?)</FIRST>', doc)
        second_match = re.search(r'<SECOND>(.*?)</SECOND>', doc)
        head_matches = re.findall(r'<HEAD>(.*?)</HEAD>', doc)
        byline_match = re.search(r'<BYLINE>(.*?)</BYLINE>', doc)
        dateline_match = re.search(r'<DATELINE>(.*?)</DATELINE>', doc)
        text_match = re.search(r'<TEXT>(.*?)</TEXT>', doc, flags=re.DOTALL)

        # Create a dictionary for each document
        doc_data = {
            'DOCNO': docno_match.group(1) if docno_match else None,
            'FILEID': fileid_match.group(1) if fileid_match else None,
            'FIRST': first_match.group(1) if first_match else None,
            'SECOND': second_match.group(1) if second_match else None,
            'HEAD': head_matches if head_matches else None,
            'BYLINE': byline_match.group(1) if byline_match else None,
            'DATELINE': dateline_match.group(1) if dateline_match else None,
            'TEXT': text_match.group(1) if text_match else None
        }

        # Append the dictionary to the data list
        data.append(doc_data)

# Create a Pandas DataFrame from the list of dictionaries
df = pd.DataFrame(data)

# Display the DataFrame
df


,DOCNO,FILEID,FIRST,SECOND,HEAD,BYLINE,DATELINE,TEXT
0,AP891220-0001,AP-NR-12-20-89 2343EST,u i BC-Panama-Resistance 12-20 0723,"BC-Panama-Resistance,0746","[Defense Forces, Noriega Resist; Loyalty Remai...",By ELOY O. AGUILAR,"PANAMA CITY, Panama (AP)",\n Instead of collapsing when the United\nSt...
1,AP891220-0002,AP-NR-12-20-89 0031EST,r a AM-RollingStones 12-20 0372,"AM-Rolling Stones,0385",[Stones Pay Per View Concert Tops North Americ...,By HENRY STERN,"ATLANTIC CITY, N.J. (AP)",\n The Rolling Stones capped the\nnext-to-la...
2,AP891220-0003,AP-NR-12-20-89 0038EST,r a PM-Christmas-Jesus'KinI 12-20 1240,"PM-Christmas-Jesus' Kin I,1277",[Part I: Jesus Descended From Checkered Ancest...,By GEORGE W. CORNELL,None,\n This first installment of a three-part Ch...
3,AP891220-0004,AP-NR-12-20-89 0039EST,r i PM-SAfrica-CoupTrial 12-20 0176,"PM-SAfrica-Coup Trial,0178",[Leader Of Failed Homeland Coup Gets 18-Year P...,None,"JOHANNESBURG, South Africa (AP)",\n A soldier who led a failed\ncoup attempt ...
4,AP891220-0005,AP-NR-12-20-89 0039EST,r a PM-RobinsonProfile 12-20 0558,"PM-Robinson Profile,0576","[Savannah Leaders Praise Slain Alderman, Eds: ...",By LAURAN NEERGAARD,"SAVANNAH, Ga. (AP)","\n Robert Robinson, the alderman and lawyer\..."
...,...,...,...,...,...,...,...,...
84674,AP890726-0308,AP-NR-07-26-89 1933EDT,u f AM-Psychiatrist-InsiderTrading 07-26 0312,"AM-Psychiatrist-Insider Trading,0322",[Psychiatrist Charged With Using Inside Inform...,By VERA HALLER,NEW YORK (AP),\n A psychiatrist was indicted Wednesday on\...
84675,AP890726-0309,AP-NR-07-26-89 1937EDT,u f AM-ComputerVirus 2ndLd-Writethru f0258 07-...,"AM-Computer Virus, 2nd Ld-Writethru, f0258,0664",[Grand Jury Indicts Cornell Student on Compute...,By JAMES ROWLEY,WASHINGTON (AP),\n A Cornell University graduate student was...
84676,AP890726-0310,AP-NR-07-26-89 2131EDT,u f AM-Japan-Markets 07-26 0275,"AM-Japan-Markets,0285","[Dollar Falls, Stock Prices Move For New High ...",None,TOKYO (AP),\n The U.S. dollar fell in early trading aga...
84677,AP890726-0311,AP-NR-07-26-89 2223EDT,r f AM-InteriorBill 3rdLd-Writethru f0329 07-2...,"AM-Interior Bill, 3rd Ld-Writethru, f0329,0762",[Senate Mulls Spending Bill Easing Ocean Drill...,By ALAN FRAM,WASHINGTON (AP),\n The Senate approved legislation on Wednes...


In [7]:
## Remove stopwords and punctuation from the text
import string
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

punctuations = string.punctuation

def stem_text(text):
    tokens = word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    tokens = [token for token in tokens if token not in list(punctuations)]
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    text = ' '.join(tokens)
    return text

def token_lengths(text):
    tokens = word_tokenize(text)
    return len(tokens)

df['PROCESSED_TEXT'] = df['TEXT'].parallel_apply(stem_text)
df['DOCNO'] = df['DOCNO'].str.strip()
df.head()


INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


,DOCNO,FILEID,FIRST,SECOND,HEAD,BYLINE,DATELINE,TEXT,PROCESSED_TEXT
0,AP891220-0001,AP-NR-12-20-89 2343EST,u i BC-Panama-Resistance 12-20 0723,"BC-Panama-Resistance,0746","[Defense Forces, Noriega Resist; Loyalty Remai...",By ELOY O. AGUILAR,"PANAMA CITY, Panama (AP)",\n Instead of collapsing when the United\nSt...,instead of collaps when the unit state threw i...
1,AP891220-0002,AP-NR-12-20-89 0031EST,r a AM-RollingStones 12-20 0372,"AM-Rolling Stones,0385",[Stones Pay Per View Concert Tops North Americ...,By HENRY STERN,"ATLANTIC CITY, N.J. (AP)",\n The Rolling Stones capped the\nnext-to-la...,the roll stone cap the next-to-last show of th...
2,AP891220-0003,AP-NR-12-20-89 0038EST,r a PM-Christmas-Jesus'KinI 12-20 1240,"PM-Christmas-Jesus' Kin I,1277",[Part I: Jesus Descended From Checkered Ancest...,By GEORGE W. CORNELL,None,\n This first installment of a three-part Ch...,thi first instal of a three-part christma seri...
3,AP891220-0004,AP-NR-12-20-89 0039EST,r i PM-SAfrica-CoupTrial 12-20 0176,"PM-SAfrica-Coup Trial,0178",[Leader Of Failed Homeland Coup Gets 18-Year P...,None,"JOHANNESBURG, South Africa (AP)",\n A soldier who led a failed\ncoup attempt ...,a soldier who led a fail coup attempt in one o...
4,AP891220-0005,AP-NR-12-20-89 0039EST,r a PM-RobinsonProfile 12-20 0558,"PM-Robinson Profile,0576","[Savannah Leaders Praise Slain Alderman, Eds: ...",By LAURAN NEERGAARD,"SAVANNAH, Ga. (AP)","\n Robert Robinson, the alderman and lawyer\...",robert robinson the alderman and lawyer assass...


In [8]:
configurations = {
    "settings" : {
        "number_of_shards": 1,
        "number_of_replicas": 1,
        "analysis": {
            "filter": {
                "english_stop": {
                    "type": "stop",
                    "stopwords_path": "my_stoplist.txt"
                }
            },
            "analyzer": {
                "stopped": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "english_stop"
                    ]
                }
            }
      }
    },
    "mappings": {
        "properties": {
            "content": {
                "type": "text",
                "fielddata": True,
                "analyzer": "stopped",
                "index_options": "positions"
            }
        }
    }
}

es.indices.create(index=index_name, body=configurations)

def add_data(id, text):
    es.index(index=index_name, id=id, document={'content': text})

# Loop through all documents
for index, row in df.iterrows():
    # Add document to the index
    add_data(row['DOCNO'], row['PROCESSED_TEXT'])


/var/folders/cg/pnd4sxn93218rhbf3zhwszv00000gn/T/ipykernel_59700/445427168.py:36: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=index_name, body=configurations)
